In [53]:
# # %pip install swifter

# import pandas as pd
# import ast
# import re
# from rapidfuzz import process as rapidfuzz_process  # Faster fuzzy matching
# import swifter  # Parallelized .apply
# from functools import lru_cache

# # --- Step 0: Load data ---
# df = pd.read_csv("datasets/foodstruct_nutritional_facts.csv", encoding="utf-8", encoding_errors="replace") 
# df_recipe = pd.read_csv("datasets/recipes_with_images.csv", encoding="utf-8", encoding_errors="replace") 




In [54]:
# # --- Step 1: Parse ingredient lists safely ---
# def parse_ingredient_list(ingredient_str):
#     if isinstance(ingredient_str, str):
#         try:
#             return ast.literal_eval(ingredient_str)
#         except Exception as e:
#             print(f"Failed to parse ingredient list: {ingredient_str} — {e}")
#             return []
#     return []

# df_recipe['Cleaned_Ingredients'] = df_recipe['Cleaned_Ingredients'].apply(parse_ingredient_list)


# # --- Step 2: Clean ingredient strings ---
# common_patterns = [
#     r"\b(?:peeled|chopped|smashed|halved|drained|optional|divided|reserved|seeded|grated|discarded|cut(?: into)?(?:.*)?|coarsely|finely|room temperature|thinly(?:.*)?|loosely packed|plus extra(?:.*)?)\b",
#     r"\([^)]*\)",  # remove anything in parentheses
#     r"\d+[\/\d]*",  # remove digits and fractions
#     r"\b(?:teaspoons?|tablespoons?|cups?|pounds?|ounces?|cloves?|slices?|cans?|sticks?|pieces?|bags?|packages?)\b",
#     r"\b(?:fresh|small|large|medium|extra-virgin|unsalted|dried|sweet|hot|cold|frozen|bottled|baby|thin|coarse|fine|creamy|whole|firm|boneless|skinless|crusty|chilled|shredded|meat|with|from|of|and|in|to|for|not|very|cut|or|into|on|at|the|a|an)\b",
# ]

# def clean_ingredient(ingredient):
#     ingredient = ingredient.lower()
#     for pattern in common_patterns:
#         ingredient = re.sub(pattern, '', ingredient)
#     ingredient = re.sub(r'[^a-z\s]', '', ingredient)  # Remove punctuation/numbers
#     ingredient = ' '.join(ingredient.split())         # Normalize whitespace
#     return ingredient.strip()

# # --- Step 3: Prepare fuzzy matching ---
# nutrient_foods = df['Food Name'].str.lower().tolist()
# skip_ingredients = {'salt', 'pepper', 'water', 'oil'}

# @lru_cache(maxsize=None)
# def fuzzy_match(ingredient):
#     if ingredient in skip_ingredients or len(ingredient) < 4:
#         return None
#     match = rapidfuzz_process.extractOne(ingredient, nutrient_foods)
#     if match:
#         name, score, _ = match
#         return name if score > 85 else None
#     return None

# unique_ingredients = {
#     clean_ingredient(raw)
#     for ingredient_list in df_recipe['Cleaned_Ingredients']
#     for raw in ingredient_list
#     if clean_ingredient(raw) not in skip_ingredients
# }

# ingredient_to_food = {ing: fuzzy_match(ing) for ing in unique_ingredients}

# # --- Step 4: Calculate nutrients ---
# nutrient_columns = df.columns.difference(['Food Name', 'Category Name'])
# def average_duplicate_foods(df):
#     df['food_name_lower'] = df['Food Name'].str.lower()
#     grouped = df.groupby('food_name_lower', as_index=False)

#     # Average nutrient values across duplicates, keep first for non-nutrient fields
#     df_avg = grouped[nutrient_columns].mean()
#     df_meta = grouped[['Food Name', 'Category Name']].first()

#     # Merge averaged nutrients with metadata
#     df_cleaned = pd.merge(df_meta, df_avg, on='food_name_lower')
#     return df_cleaned.set_index('food_name_lower')

# # Create cleaned and indexed food dataframe
# df_indexed = average_duplicate_foods(df)

# # Optional: print how many duplicates were averaged
# num_duplicates = df['food_name_lower'].duplicated().sum()
# if num_duplicates > 0:
#     print(f"✅ Averaged {num_duplicates} duplicate food entries.")

# unique_nutrient_errors = set()

# def calculate_nutrients(ingredient_list):
#     if not isinstance(ingredient_list, list):
#         return pd.Series(0.0, index=nutrient_columns)

#     total = pd.Series(0.0, index=nutrient_columns)

#     for raw in ingredient_list:
#         if not isinstance(raw, str):
#             continue
#         cleaned = clean_ingredient(raw)
#         food_name = ingredient_to_food.get(cleaned)
#         if isinstance(food_name, str) and food_name in df_indexed.index:
#             match_row = df_indexed.loc[food_name]
#             if isinstance(match_row, pd.DataFrame):
#                 match_row = match_row.iloc[0]  # Take the first row if duplicate
#             try:
#                 total += match_row[nutrient_columns]
#             except Exception as e:
#                 error_msg = f"Error adding nutrients for '{food_name}': {e}"
#                 unique_nutrient_errors.add(error_msg)
#                 continue

#     return total

# # --- Step 5: Calculate nutrients per recipe ---
# df_nutrients = df_recipe['Cleaned_Ingredients'].apply(calculate_nutrients)

# # --- Step 6: Print any errors ---
# if unique_nutrient_errors:
#     print("\nUnique errors during nutrient addition:")
#     for err in unique_nutrient_errors:
#         print(err)


# # --- Step 7: Combine and save ---
# df_recipe_total = pd.concat([df_recipe, df_nutrients], axis=1)

# # --- Step 8: Split into Macro and Micro Nutrients ---
# macro_cols = [
#     'Carbs', 'Fats', 'Fiber', 'Net carbs', 'Protein',
#     'Saturated Fat', 'Monounsaturated Fat', 'Polyunsaturated fat',
#     'Trans Fat', 'Sugar', 'Starch'
# ]

# micro_cols = [col for col in df_nutrients.columns if col not in macro_cols and col != 'Calories']

# # Create new columns as arrays of (name, value) tuples
# df_recipe_total['Macro_Nutrients'] = df_recipe_total.apply(
#     lambda row: [(col, row[col]) for col in macro_cols if col in row], axis=1
# )

# df_recipe_total['Micro_Nutrients'] = df_recipe_total.apply(
#     lambda row: [(col, row[col]) for col in micro_cols if col in row], axis=1
# )

# # Drop individual nutrient columns (except Calories, if you want to keep it)
# cols_to_keep = [
#     'Title', 'Ingredients', 'Instructions', 'Image_Name', 'Cleaned_Ingredients',
#     'Calories', 'Macro_Nutrients', 'Micro_Nutrients'
# ]
# df_recipe_total = df_recipe_total[cols_to_keep]

# df_recipe_total.to_csv("datasets/recipes_with_images_and_nutrients.csv", index=False)

In [55]:
# df.to_markdown

In [56]:
# df_recipe.to_markdown

In [57]:
# # --- Step 1: Parse ingredient lists safely ---
# def parse_ingredient_list(ingredient_str):
#     if isinstance(ingredient_str, str):
#         try:
#             return ast.literal_eval(ingredient_str)
#         except Exception as e:
#             print(f"Failed to parse ingredient list: {ingredient_str} — {e}")
#             return []
#     return []

# df_recipe['Cleaned_Ingredients'] = df_recipe['Cleaned_Ingredients'].apply(parse_ingredient_list)


# # --- Step 2: Clean ingredient strings ---
# common_patterns = [
#     r"\b(?:peeled|chopped|smashed|halved|drained|optional|divided|reserved|seeded|grated|discarded|cut(?: into)?(?:.*)?|coarsely|finely|room temperature|thinly(?:.*)?|loosely packed|plus extra(?:.*)?)\b",
#     r"\([^)]*\)",  # remove anything in parentheses
#     r"\d+[\/\d]*",  # remove digits and fractions
#     r"\b(?:teaspoons?|tablespoons?|cups?|pounds?|ounces?|cloves?|slices?|cans?|sticks?|pieces?|bags?|packages?)\b",
#     r"\b(?:fresh|small|large|medium|extra-virgin|unsalted|dried|sweet|hot|cold|frozen|bottled|baby|thin|coarse|fine|creamy|whole|firm|boneless|skinless|crusty|chilled|shredded|meat|with|from|of|and|in|to|for|not|very|cut|or|into|on|at|the|a|an)\b",
# ]

# def clean_ingredient(ingredient):
#     ingredient = ingredient.lower()
#     for pattern in common_patterns:
#         ingredient = re.sub(pattern, '', ingredient)
#     ingredient = re.sub(r'[^a-z\s]', '', ingredient)  # Remove punctuation/numbers
#     ingredient = ' '.join(ingredient.split())         # Normalize whitespace
#     return ingredient.strip()

# # --- Step 3: Prepare fuzzy matching ---
# nutrient_foods = df['Food Name'].str.lower().tolist()
# skip_ingredients = {'salt', 'pepper', 'water', 'oil'}

# @lru_cache(maxsize=None)
# def fuzzy_match(ingredient):
#     if ingredient in skip_ingredients or len(ingredient) < 4:
#         return None
#     match = rapidfuzz_process.extractOne(ingredient, nutrient_foods)
#     if match:
#         name, score, _ = match
#         return name if score > 85 else None
#     return None

# unique_ingredients = {
#     clean_ingredient(raw)
#     for ingredient_list in df_recipe['Cleaned_Ingredients']
#     for raw in ingredient_list
#     if clean_ingredient(raw) not in skip_ingredients
# }

# ingredient_to_food = {ing: fuzzy_match(ing) for ing in unique_ingredients}

# # --- Step 4: Calculate nutrients ---
# nutrient_columns = df.columns.difference(['Food Name', 'Category Name'])
# def average_duplicate_foods(df):
#     df['food_name_lower'] = df['Food Name'].str.lower()
#     grouped = df.groupby('food_name_lower', as_index=False)

#     # Average nutrient values across duplicates, keep first for non-nutrient fields
#     df_avg = grouped[nutrient_columns].mean()
#     df_meta = grouped[['Food Name', 'Category Name']].first()

#     # Merge averaged nutrients with metadata
#     df_cleaned = pd.merge(df_meta, df_avg, on='food_name_lower')
#     return df_cleaned.set_index('food_name_lower')

# # Create cleaned and indexed food dataframe
# df_indexed = average_duplicate_foods(df)

# # Optional: print how many duplicates were averaged
# num_duplicates = df['food_name_lower'].duplicated().sum()
# if num_duplicates > 0:
#     print(f"✅ Averaged {num_duplicates} duplicate food entries.")

# unique_nutrient_errors = set()

# def calculate_nutrients(ingredient_list):
#     if not isinstance(ingredient_list, list):
#         return pd.Series(0.0, index=nutrient_columns)

#     total = pd.Series(0.0, index=nutrient_columns)

#     for raw in ingredient_list:
#         if not isinstance(raw, str):
#             continue
#         cleaned = clean_ingredient(raw)
#         food_name = ingredient_to_food.get(cleaned)
#         if isinstance(food_name, str) and food_name in df_indexed.index:
#             match_row = df_indexed.loc[food_name]
#             if isinstance(match_row, pd.DataFrame):
#                 match_row = match_row.iloc[0]  # Take the first row if duplicate
#             try:
#                 total += match_row[nutrient_columns]
#             except Exception as e:
#                 error_msg = f"Error adding nutrients for '{food_name}': {e}"
#                 unique_nutrient_errors.add(error_msg)
#                 continue

#     return total

# # --- Step 5: Calculate nutrients per recipe ---
# df_nutrients = df_recipe['Cleaned_Ingredients'].apply(calculate_nutrients)

# # --- Step 6: Print any errors ---
# if unique_nutrient_errors:
#     print("\nUnique errors during nutrient addition:")
#     for err in unique_nutrient_errors:
#         print(err)


# # --- Step 7: Combine and save ---
# df_recipe_total = pd.concat([df_recipe, df_nutrients], axis=1)

# # --- Step 8: Split into Macro and Micro Nutrients ---
# macro_cols = [
#     'Carbs', 'Fats', 'Fiber', 'Net carbs', 'Protein',
#     'Saturated Fat', 'Monounsaturated Fat', 'Polyunsaturated fat',
#     'Trans Fat', 'Sugar', 'Starch'
# ]

# micro_cols = [col for col in df_nutrients.columns if col not in macro_cols and col != 'Calories']

# # Create new columns as arrays of (name, value) tuples
# df_recipe_total['Macro_Nutrients'] = df_recipe_total.apply(
#     lambda row: [(col, row[col]) for col in macro_cols if col in row], axis=1
# )

# df_recipe_total['Micro_Nutrients'] = df_recipe_total.apply(
#     lambda row: [(col, row[col]) for col in micro_cols if col in row], axis=1
# )

# # Drop individual nutrient columns (except Calories, if you want to keep it)
# cols_to_keep = [
#     'Title', 'Ingredients', 'Instructions', 'Image_Name', 'Cleaned_Ingredients',
#     'Calories', 'Macro_Nutrients', 'Micro_Nutrients'
# ]
# df_recipe_total = df_recipe_total[cols_to_keep]

# df_recipe_total.to_csv("datasets/recipes_with_images_and_nutrients.csv", index=False)

In [58]:
# import pandas as pd

# # Load the uploaded CSV files
# recipes_df = pd.read_csv("datasets/recipes_with_images.csv")
# nutrition_df = pd.read_csv("datasets/foodstruct_nutritional_facts.csv")

# # Display the first few rows of each to understand their structure
# recipes_head = recipes_df.head()
# nutrition_head = nutrition_df.head()

# recipes_head, nutrition_head


In [59]:
# # Preview a few cleaned ingredient lists to understand formatting
# sample_ingredients = recipes_df['Cleaned_Ingredients'].head(5).tolist()

# # Print the first recipe's cleaned ingredients
# sample_ingredients[1]



In [60]:

# import ast

# # Convert the string representation of a list into an actual list for the first recipe
# first_recipe_ingredients = ast.literal_eval(sample_ingredients[1])

# # Show the parsed list of ingredients
# first_recipe_ingredients


In [61]:
# import re
# from fractions import Fraction
# from unidecode import unidecode
# import pandas as pd

# unit_to_grams = {
#     "tsp": 4.2,
#     "tbsp": 14.0,
#     "cup": 240.0,
#     "oz": 28.35,
#     "lb": 453.6,
#     "loaf": 450.0,
#     "medium": 150.0,
#     "small": 100.0,
#     "pinch": 0.36,
#     "piece": 50.0,
# }

# # Common preparation descriptors to remove (extended with your skip_descriptors)
# skip_descriptors = {
#     "melted", "crushed", "finely chopped", "chopped",
#     "diced", "sliced", "room temperature", "minced",
#     "whole", "good quality", "sturdy", "freshly ground",
#     "freshly", "ground", "of", "about", "total"
# }

# def remove_descriptive_phrases(ingredient):
#     # Patterns to cut off descriptive text
#     cutoff_phrases = [
#         r'\bsuch as\b',
#         r'\blike\b',
#         r'\bfor example\b',
#         r'\bincluding\b',
#         r'\b(e\.g\.)\b',
#         r'\(.*\)',  # Also remove any parentheses content as a catch-all
#     ]

#     ingr = ingredient.lower()
#     for phrase in cutoff_phrases:
#         ingr = re.split(phrase, ingr)[0]  # Take text before phrase only

#     ingr = ingr.strip()
#     return ingr

# def clean_ingredient(ingredient):
#     # Lowercase
#     ingr = ingredient.lower()

#     # Replace all dash types with a space
#     for dash_char in ['–', '−', '—', '-']:
#         ingr = ingr.replace(dash_char, ' ')

#     # Remove skip descriptors (whole words or phrases)
#     for phrase in sorted(skip_descriptors, key=len, reverse=True):
#         pattern = r'\b' + re.escape(phrase) + r'\b'
#         ingr = re.sub(pattern, '', ingr)

#     # Remove leftover numbers, fractions, and punctuation that are not part of ingredient name
#     ingr = re.sub(r'\b[\d¼½¾⅓⅔⅛\/\.\,]+\b', '', ingr)

#     # Remove leftover units (if any remain)
#     units = ["tsp", "tbsp", "cup", "oz", "lb", "loaf", "medium", "small", "pinch", "piece"]
#     for unit in units:
#         ingr = re.sub(r'\b' + re.escape(unit) + r's?\b', '', ingr)

#     # Remove extra punctuation and multiple spaces
#     ingr = re.sub(r'[^\w\s]', '', ingr)
#     ingr = re.sub(r'\s+', ' ', ingr)

#     # Strip leading/trailing whitespace
#     ingr = ingr.strip()

#     return ingr


# def parse_fraction_to_float(s):
#     s = s.strip()
#     if not s:
#         return 1.0
#     # Handle known unicode fractions in combination with whole numbers (e.g., 1½)
#     mixed_frac_match = re.match(r"(\d+)([¼½¾⅓⅔⅛]+)", s)
#     if mixed_frac_match:
#         whole = int(mixed_frac_match.group(1))
#         frac_part = mixed_frac_match.group(2)
#         frac_map = {
#             '¼': Fraction(1, 4),
#             '½': Fraction(1, 2),
#             '¾': Fraction(3, 4),
#             '⅓': Fraction(1, 3),
#             '⅔': Fraction(2, 3),
#             '⅛': Fraction(1, 8),
#         }
#         frac_value = sum(frac_map.get(ch, 0) for ch in frac_part)
#         return whole + float(frac_value)

#     # Try to parse numeric parts safely
#     parts = s.split()
#     total = 0
#     for part in parts:
#         part = part.strip()
#         if part in {'.', '-', '/', ''}:
#             continue
#         try:
#             total += float(Fraction(part))
#         except (ValueError, ZeroDivisionError):
#             continue
#     return total if total != 0 else 1.0


# def sort_units_with_priority(units, priority_order):
#     prioritized, seen = [], set()
#     for group in priority_order:
#         for u in group:
#             if u in units:
#                 prioritized.append(u)
#                 seen.add(u)
#     remaining = [u for u in units if u not in seen]
#     prioritized.extend(sorted(remaining, key=len, reverse=True))
#     return prioritized


# def find_all_qty_units(text, pattern_units):
#     """Find all (qty, unit, position) in text, including inside parentheses."""
#     regex = re.compile(rf"([\d\s\/\.\¼\½\¾\⅓\⅔\⅛\-]+)?\s*({pattern_units})\b")
#     matches = []
#     for m in regex.finditer(text):
#         qty_str = m.group(1)
#         unit = m.group(2)
#         start = m.start()
#         matches.append((qty_str, unit, start))
#     return matches


# def unit_priority(unit, priority_order):
#     """Return priority index of unit based on priority_order."""
#     for i, group in enumerate(priority_order):
#         if unit in group:
#             return i
#     return len(priority_order)  # lowest priority if not found


# def parse_quantity_and_unit(text):
#     text = unidecode(text).lower()
#     for ch in ['–', '−', '—']:
#         text = text.replace(ch, '-')

#     units = list(unit_to_grams.keys())
#     priority_order = [['lb', 'oz'], ['tsp', 'tbsp', 'cup'], ['loaf', 'medium', 'small', 'pinch', 'piece']]
#     units = sort_units_with_priority(units, priority_order)
#     pattern_units = '|'.join(units)

#     text = re.sub(r'-(?=\s*(' + pattern_units + r'))', ' ', text)

#     matches = find_all_qty_units(text, pattern_units)
#     if not matches:
#         return None

#     # Parse quantities and assign priorities
#     parsed_matches = []
#     for qty_str, unit, pos in matches:
#         try:
#             if qty_str:
#                 qty_str = qty_str.strip().rstrip('-')
#                 if '-' in qty_str:
#                     low, high = map(parse_fraction_to_float, qty_str.split('-')[:2])
#                     quantity = (low + high) / 2
#                 else:
#                     quantity = parse_fraction_to_float(qty_str)
#             else:
#                 quantity = 1.0
#         except Exception:
#             quantity = 1.0

#         prio = unit_priority(unit, priority_order)
#         parsed_matches.append((quantity, unit, pos, prio))

#     # Sort by priority (lowest index = highest priority), then by position in text
#     parsed_matches.sort(key=lambda x: (x[3], x[2]))

#     # Select highest priority match
#     quantity, unit, pos, prio = parsed_matches[0]

#     # Remove only the matched qty and unit from text once
#     # Build regex pattern for matched qty+unit substring for removal
#     qty_pattern = rf"[\d\s\/\.\¼\½\¾\⅓\⅔\⅛\-]*"
#     # Escape unit in regex
#     escaped_unit = re.escape(unit)
#     removal_pattern = re.compile(rf"{qty_pattern}{escaped_unit}\b")
#     ingredient = removal_pattern.sub('', text, count=1).strip()

#     return quantity, unit, ingredient


# def parse_and_merge_quantities(text):
#     parts = re.split(r'\bplus\b', text, flags=re.IGNORECASE)
#     merged = {
#         "quantity": 0.0,
#         "unit": None,
#     }

#     base_ingredient = None

#     for part in parts:
#         part = part.strip()
#         result = parse_quantity_and_unit(part)
#         if result is None:
#             continue
#         qty, unit, ingredient = result

#         # Skip generic additions like "plus more"
#         if qty == 1.0 and unit == "unit" and ingredient in {"more", "to taste", "as needed"}:
#             continue

#         if not isinstance(qty, (float, int)):
#             qty = 1.0

#         if not merged["unit"]:
#             merged["unit"] = unit

#         ingredient_parts = [p.strip() for p in ingredient.split(',')]
#         name = ingredient_parts[0]

#         if not base_ingredient:
#             base_ingredient = name

#         merged["quantity"] += qty

#     merged["ingredient"] = base_ingredient
#     return merged


# # Assuming first_recipe_ingredients is a list of ingredient strings
# parsed_ingredients = []
# for item in first_recipe_ingredients:
#     merged = parse_and_merge_quantities(item)
#     if merged is None or merged["quantity"] == 0:
#         continue

#     qty = merged["quantity"]
#     unit = merged["unit"]
#     ingredient = merged["ingredient"]

#     # Clean the ingredients
#     cleaned_ingredient = clean_ingredient(ingredient)

#     # Remove descriptive phrases like 'such as gala or pink lady'
#     main_ingredient = remove_descriptive_phrases(cleaned_ingredient)

#     grams = qty * unit_to_grams.get(unit, 1.0)
#     parsed_ingredients.append({
#         "original": item,
#         "quantity": qty,
#         "unit": unit,
#         "ingredient": main_ingredient,
#         "grams": grams
#     })

# df = pd.DataFrame(parsed_ingredients)
# df

In [62]:
# # Assuming:
# # df = your parsed ingredients DataFrame with columns: ['original', 'quantity', 'unit', 'ingredient', 'grams']
# # nutrition_df = DataFrame with nutrition per 100g and a 'Food Name' column

# # First, prepare a list of nutrient columns to sum
# nutrient_columns = [
#     "Calcium","Calories","Carbs","Cholesterol","Copper","Fats","Fiber","Folate",
#     "Iron","Magnesium","Monounsaturated Fat","Net carbs","Omega-3 - DHA","Omega-3 - DPA",
#     "Omega-3 - EPA","Phosphorus","Polyunsaturated fat","Potassium","Protein",
#     "Saturated Fat","Selenium","Sodium","Trans Fat","Vitamin A (IU)","Vitamin A RAE",
#     "Vitamin B1","Vitamin B12","Vitamin B2","Vitamin B3","Vitamin B5","Vitamin B6",
#     "Vitamin C","Zinc","Choline","Fructose","Histidine","Isoleucine","Leucine",
#     "Lysine","Manganese","Methionine","Phenylalanine","Starch","Sugar","Threonine",
#     "Tryptophan","Valine","Vitamin D","Vitamin E","Vitamin K","Omega-3 - ALA",
#     "Omega-6 - Eicosadienoic acid","Omega-6 - Gamma-linoleic acid",
#     "Omega-3 - Eicosatrienoic acid","Omega-6 - Dihomo-gamma-linoleic acid",
#     "Omega-6 - Linoleic acid","Omega-6 - Arachidonic acid"
# ]

# # Create a dict to accumulate totals
# total_nutrients = {nutrient: 0.0 for nutrient in nutrient_columns}

# # Normalize ingredient names for matching (lowercase, strip)
# nutrition_df['Food Name Clean'] = nutrition_df['Food Name'].str.lower().str.strip()
# df['ingredient_clean'] = df['ingredient'].str.lower().str.strip()

# for idx, row in df.iterrows():
#     ingredient_name = row['ingredient_clean']
#     grams = row['grams']

#     # Standardize ingredient names
#     if ingr_name == "chicken":
#         ingr_name = "chicken meat"
#     elif ingr_name == "unsalted butter":
#         ingr_name = "butter"
#     elif ingr_name == "cooked black beans":
#         ingr_name = "black turtle bean"

#     # Attempt direct match
#     nutrition_row = nutrition_df[nutrition_df['Food Name Clean'] == ingr_name]

#     # Fallback: remove trailing 's' if present
#     if nutrition_row.empty and ingr_name.endswith('s'):
#         alt_ingr_name = ingr_name[:-1]
#         nutrition_row = nutrition_df[nutrition_df['Food Name Clean'] == alt_ingr_name]
#         if not nutrition_row.empty:
#             print(f"Fallback match used: '{ingr_name}' ➜ '{alt_ingr_name}'")
#             ingr_name = alt_ingr_name

#     if nutrition_row.empty:
#         print(f"No nutrition data found for ingredient: '{ingr_name}'")
#         continue


#     # Nutrients per 100g
#     nutrients_per_100g = nutrition_row.iloc[0][nutrient_columns]

#     # Scale nutrients by grams / 100
#     nutrients_scaled = nutrients_per_100g * (grams / 100.0)

#     # Sum nutrients
#     for nutrient in nutrient_columns:
#         val = nutrients_scaled.get(nutrient, 0)
#         if pd.notna(val):
#             total_nutrients[nutrient] += val

# # Prepare the final DataFrame with the recipe name and summed nutrients
# recipe_name = "Your Recipe Name"  # Change this accordingly

# result_dict = {"Recipe Name": recipe_name}
# result_dict.update(total_nutrients)

# result_df = pd.DataFrame([result_dict])

# result_df

In [70]:
import pandas as pd
import re
from fractions import Fraction
from unidecode import unidecode
import ast

# === Load the data ===
recipes_df = pd.read_csv("datasets/recipes_with_images.csv")
nutrition_df = pd.read_csv("datasets/foodstruct_nutritional_facts.csv")

# === Constants and Helper Functions ===

unit_to_grams = {
    "tsp": 4.2, "tbsp": 14.0, "cup": 240.0, "oz": 28.35,
    "lb": 453.6, "loaf": 450.0, "medium": 150.0,
    "small": 100.0, "pinch": 0.36, "piece": 50.0,
}

skip_descriptors = {
    "melted", "crushed", "finely chopped", "chopped",
    "diced", "sliced", "room temperature", "minced",
    "whole", "good quality", "sturdy", "freshly ground",
    "freshly", "ground", "of", "about", "total", "caster", 
    "granulated", "superfine", "ripe", "very ripe", "caster granulated",
    "very", "thinly", "thin", "to", "sticks", "unrefined", "unripe", 
    "unpeeled", "center", "cut", "uncooked", "trimmed", "torn",
    "puree", "toasted", "tightly packed", "x", "wild", "well",
    "stirred", "shaken", "cooked", "canned", "wheat", "your", 
    "unseasoned", "favorite", "you", "will", "need", "unsifted", "sifted",
    "unsalted", "unflavored", "unhusked", "unpacked", "unpasteurized", "pasteurized",
    "unbleached", "with", "added", "from", "can", "canned", "leaves", "thick", "thawed",
    "yellow", "white", "optional", "halves" "half", "ripened", "unblanched", "unfiltered", 
    "blanched", "filtered", "uncle bens", "grated", "shredded", "untrimmed", "untoasted", "unsulfured",
    "underripe", "unhulled", "unsliced", "ultrathin", "slices", "wrapped", "washed", "warm", "unsweetened", "at", "hour"
}

def remove_descriptive_phrases(ingredient):
    cutoff_phrases = [r'\bsuch as\b', r'\blike\b', r'\bfor example\b', r'\bincluding\b', r'\b(e\.g\.)\b', r'\(.*\)']
    ingr = ingredient.lower()
    for phrase in cutoff_phrases:
        ingr = re.split(phrase, ingr)[0]
    return ingr.strip()

def clean_ingredient(ingredient):
    ingr = ingredient.lower()
    for dash_char in ['–', '−', '—', '-']:
        ingr = ingr.replace(dash_char, ' ')
    for phrase in sorted(skip_descriptors, key=len, reverse=True):
        pattern = r'\b' + re.escape(phrase) + r'\b'
        ingr = re.sub(pattern, '', ingr)
    ingr = re.sub(r'\b[\d¼½¾⅓⅔⅛\/\.\,]+\b', '', ingr)
    for unit in unit_to_grams.keys():
        ingr = re.sub(r'\b' + re.escape(unit) + r's?\b', '', ingr)
    ingr = re.sub(r'[^\w\s]', '', ingr)
    ingr = re.sub(r'\s+', ' ', ingr).strip()
    return ingr

def parse_fraction_to_float(s):
    s = s.strip()
    if not s:
        return 1.0
    # Handle known unicode fractions in combination with whole numbers (e.g., 1½)
    mixed_frac_match = re.match(r"(\d+)([¼½¾⅓⅔⅛]+)", s)
    if mixed_frac_match:
        whole = int(mixed_frac_match.group(1))
        frac_part = mixed_frac_match.group(2)
        frac_map = {
            '¼': Fraction(1, 4),
            '½': Fraction(1, 2),
            '¾': Fraction(3, 4),
            '⅓': Fraction(1, 3),
            '⅔': Fraction(2, 3),
            '⅛': Fraction(1, 8),
        }
        frac_value = sum(frac_map.get(ch, 0) for ch in frac_part)
        return whole + float(frac_value)

    # Try to parse numeric parts safely
    parts = s.split()
    total = 0
    for part in parts:
        part = part.strip()
        if part in {'.', '-', '/', ''}:
            continue
        try:
            total += float(Fraction(part))
        except (ValueError, ZeroDivisionError):
            continue
    return total if total != 0 else 1.0

def sort_units_with_priority(units, priority_order):
    prioritized, seen = [], set()
    for group in priority_order:
        for u in group:
            if u in units:
                prioritized.append(u)
                seen.add(u)
    remaining = [u for u in units if u not in seen]
    return prioritized + sorted(remaining, key=len, reverse=True)

def find_all_qty_units(text, pattern_units):
    return [(m.group(1), m.group(2), m.start()) for m in re.finditer(
        rf"([\d¼½¾⅓⅔⅛\/\.\,\s\-]*)(?:\s*|)(?<![a-zA-Z])({pattern_units})\b", text)]

def unit_priority(unit, priority_order):
    return next((i for i, group in enumerate(priority_order) if unit in group), len(priority_order))

def parse_quantity_and_unit(text):
    text = unidecode(text).lower().replace('–', '-').replace('−', '-').replace('—', '-')
    units = list(unit_to_grams.keys())
    priority_order = [['lb', 'oz'], ['tsp', 'tbsp', 'cup'], ['loaf', 'medium', 'small', 'pinch', 'piece']]
    units = sort_units_with_priority(units, priority_order)
    pattern_units = '|'.join(units)
    matches = find_all_qty_units(text, pattern_units)
    if not matches:
        return None
    parsed = []
    for qty_str, unit, pos in matches:
        qty_str = qty_str.strip().rstrip('-')
        quantity = (parse_fraction_to_float(qty_str.split('-')[0]) + parse_fraction_to_float(qty_str.split('-')[1])) / 2 if '-' in qty_str else parse_fraction_to_float(qty_str)
        parsed.append((quantity, unit, pos, unit_priority(unit, priority_order)))
    quantity, unit, pos, _ = sorted(parsed, key=lambda x: (x[3], x[2]))[0]
    ingredient = re.sub(rf"[\d\s\/\.\¼\½\¾\⅓\⅔\⅛\-]*{re.escape(unit)}\b", '', text, count=1).strip()
    return quantity, unit, ingredient

def parse_and_merge_quantities(text):
    parts = re.split(r'\bplus\b', text, flags=re.IGNORECASE)
    merged = {"quantity": 0.0, "unit": None}
    base_ingredient = None
    for part in parts:
        result = parse_quantity_and_unit(part.strip())
        if result:
            qty, unit, ingredient = result
            if not merged["unit"]:
                merged["unit"] = unit
            name = ingredient.split(',')[0].strip()
            if not base_ingredient:
                base_ingredient = name
            merged["quantity"] += qty
    merged["ingredient"] = base_ingredient
    return merged

# === Normalize Nutrition Data ===
nutrition_df['Food Name Clean'] = nutrition_df['Food Name'].str.lower().str.strip()

nutrient_columns = [
    "Calcium", "Calories", "Carbs", "Cholesterol", "Copper", "Fats", "Fiber", "Folate",
    "Iron", "Magnesium", "Monounsaturated Fat", "Net carbs", "Omega-3 - DHA", "Omega-3 - DPA",
    "Omega-3 - EPA", "Phosphorus", "Polyunsaturated fat", "Potassium", "Protein",
    "Saturated Fat", "Selenium", "Sodium", "Trans Fat", "Vitamin A (IU)", "Vitamin A RAE",
    "Vitamin B1", "Vitamin B12", "Vitamin B2", "Vitamin B3", "Vitamin B5", "Vitamin B6",
    "Vitamin C", "Zinc", "Choline", "Fructose", "Histidine", "Isoleucine", "Leucine",
    "Lysine", "Manganese", "Methionine", "Phenylalanine", "Starch", "Sugar", "Threonine",
    "Tryptophan", "Valine", "Vitamin D", "Vitamin E", "Vitamin K", "Omega-3 - ALA",
    "Omega-6 - Eicosadienoic acid", "Omega-6 - Gamma-linoleic acid", "Omega-3 - Eicosatrienoic acid",
    "Omega-6 - Dihomo-gamma-linoleic acid", "Omega-6 - Linoleic acid", "Omega-6 - Arachidonic acid"
]

# Hardcoded nutritional values for 100g of raw turkey meat
turkey_nutrients = {
    "Calories": 135,
    "Carbs": 0,
    "Fats": 7.0,
    "Protein": 30.0,
    "Saturated Fat": 2.0,
    "Monounsaturated Fat": 2.0,
    "Polyunsaturated fat": 1.5,
    "Trans Fat": 0.0,
    "Fiber": 0.0,
    "Sugar": 0.0,
    "Cholesterol": 70,
    "Sodium": 50,
    "Iron": 1.2,
    "Zinc": 2.0,
    "Vitamin B6": 0.6,
    "Vitamin B12": 1.0,
}


# === Process Recipes ===
def get_nutrition_row(ingredient_name: str, grams: float, nutrition_df, nutrient_columns, fallback_matches, unmatched_ingredients):
    """
    Attempts to find a matching row in the nutrition database for the given ingredient.
    If no exact match is found, tries a singular fallback (e.g., 'apples' → 'apple').

    Returns the matching row or None if no match is found.
    Updates unmatched_ingredients and fallback_matches in place.
    """
    ingr_name = ingredient_name.lower().strip()
    original_name = ingr_name

    # Common manual mappings
    manual_map = {
        "chicken": "chicken meat",
        "unsalted butter": "butter",
        "cooked black beans": "black turtle bean",
        "white wheat flour": "flour",
        "yellow onions" : "onion",
        "yellow onion" : "onion",
        "whipping cream": "whipped cream",
        "yukon gold potato": "potato",
    }
    ingr_name = manual_map.get(ingr_name, ingr_name)

    nutrition_row = nutrition_df[nutrition_df['Food Name Clean'] == ingr_name]

    # Try fallback by stripping plural forms
    if nutrition_row.empty:
        alt_ingr = None
        if ingr_name.endswith('es'):
            alt_ingr = ingr_name[:-2]
        elif ingr_name.endswith('s'):
            alt_ingr = ingr_name[:-1]
        
        if alt_ingr:
            nutrition_row = nutrition_df[nutrition_df['Food Name Clean'] == alt_ingr]
            if not nutrition_row.empty:
                fallback_matches.add((original_name, alt_ingr))
                ingr_name = alt_ingr

    if ingr_name == "turkey":
        scaled_manual = {nut: (val * grams / 100.0) for nut, val in turkey_nutrients.items()}
        for nutrient in nutrient_columns:
            if nutrient not in scaled_manual:
                scaled_manual[nutrient] = 0.0
        return pd.Series(scaled_manual)

    if nutrition_row.empty:
        unmatched_ingredients.add((original_name, grams))
        return None

    return nutrition_row.iloc[0]


# Declare sets outside loop
results = []
unmatched_ingredients = set()
fallback_matches = set()

for _, row in recipes_df.iterrows():
    title, image_name, instructions, ingredients = row['Title'], row['Image_Name'], row['Instructions'], row['Cleaned_Ingredients']
    
    try:
        ingredients_list = ast.literal_eval(ingredients)
    except Exception:
        continue

    parsed_ingredients = []
    for item in ingredients_list:
        merged = parse_and_merge_quantities(item)
        if not merged or merged["quantity"] == 0:
            continue

        qty, unit = merged["quantity"], merged["unit"]
        ingr = clean_ingredient(merged["ingredient"])
        ingr = remove_descriptive_phrases(ingr)

        # Skip ingredients that contain the word 'water'
        if any(skip_word in ingr for skip_word in ['water', 'wood chips']):
            continue

        grams = qty * unit_to_grams.get(unit, 1.0)
        parsed_ingredients.append({"quantity": qty, "unit": unit, "ingredient": ingr, "grams": grams})

    total_nutrients = {nutrient: 0.0 for nutrient in nutrient_columns}

    for ingr_data in parsed_ingredients:
        ingr_name = ingr_data['ingredient']
        grams = ingr_data['grams']

        nutrition_row = get_nutrition_row(ingr_name, grams, nutrition_df, nutrient_columns, fallback_matches, unmatched_ingredients)
        if nutrition_row is None:
            continue

        scaled = nutrition_row[nutrient_columns] * (grams / 100.0)
        for nutrient in nutrient_columns:
            if pd.notna(scaled.get(nutrient)):
                total_nutrients[nutrient] += scaled[nutrient]

    record = {
        "Recipe Name": title,
        "Ingredients": ingredients,
        "Instructions": instructions,
        "Image_Name": image_name
    }
    record.update(total_nutrients)
    results.append(record)


if unmatched_ingredients:
    print("Unmatched ingredients with quantities:")
    for ingr, qty in sorted(unmatched_ingredients):
        print(f"{ingr}: {qty:.2f} grams")
else:
    print("All ingredients matched.")

# === Final DataFrame ===
df = pd.DataFrame(results)

# After computing result_df:
macro_cols = [
    'Carbs', 'Fats', 'Fiber', 'Net carbs', 'Protein',
    'Saturated Fat', 'Monounsaturated Fat', 'Polyunsaturated fat',
    'Trans Fat', 'Sugar', 'Starch'
]
micro_cols = [col for col in df.columns if col not in macro_cols and col not in ['Calories', 'Recipe Name', 'Ingredients', 'Instructions', 'Image_Name']]

df['Micro_Nutrients'] = df.apply(
    lambda row: [
        (col, rounded) for col in micro_cols
        if col in row and (rounded := round(row[col], 2)) != 0
    ],
    axis=1
)

df['Macro_Nutrients'] = df.apply(
    lambda row: [
        (col, rounded) for col in macro_cols
        if col in row and (rounded := round(row[col], 2)) != 0
    ],
    axis=1
)

# Columns to keep
cols_to_keep = [
    'Recipe Name', 'Ingredients', 'Instructions', 'Image_Name',
    'Calories', 'Macro_Nutrients', 'Micro_Nutrients'
]
df = df[cols_to_keep]

# Remove rows with 0 Calories
df = df[df['Calories'] > 0]

df['Calories'] = df['Calories'].round(2)

# Columns to keep
cols_to_keep = [
    'Recipe Name', 'Ingredients', 'Instructions', 'Image_Name',
    'Calories', 'Macro_Nutrients', 'Micro_Nutrients'
]
df = df[cols_to_keep]

# Clean the Ingredients field: remove brackets, apostrophes, join nicely
def clean_ingredients_text(ingredients):
    try:
        ingr_list = ast.literal_eval(ingredients)
        return "\n".join(item.strip() for item in ingr_list if isinstance(item, str))
    except:
        return ingredients  # fallback if eval fails

df['Ingredients'] = df['Ingredients'].apply(clean_ingredients_text)


# Save the cleaned and enriched dataset
df.to_csv("datasets/recipes_with_images_and_nutrients.csv", index=False)


Unmatched ingredients with quantities:
: 0.36 grams
: 2.10 grams
: 6.30 grams
: 8.40 grams
: 14.00 grams
: 14.70 grams
: 16.80 grams
: 17.50 grams
: 17.85 grams
: 18.67 grams
: 21.00 grams
: 24.50 grams
: 28.00 grams
: 31.50 grams
: 32.67 grams
: 35.00 grams
: 42.00 grams
: 45.50 grams
: 49.00 grams
: 56.00 grams
: 60.00 grams
: 70.00 grams
: 80.00 grams
: 94.50 grams
: 100.00 grams
: 120.00 grams
: 150.00 grams
: 160.00 grams
: 180.00 grams
: 240.00 grams
: 300.00 grams
: 340.20 grams
: 450.00 grams
: 500.00 grams
: 540.00 grams
: 600.00 grams
: 800.00 grams
: 3600.00 grams
100g: 120.00 grams
100g all purpose flour: 99.23 grams
100g all purpose flour: 180.00 grams
100g asparagus: 28.35 grams
100g blue cheese: 99.23 grams
100g butter: 98.00 grams
100g butter: 113.40 grams
100g butter: 120.00 grams
100g chocolate: 113.40 grams
100g dark chocolate: 99.23 grams
100g dark chocolate cacao: 99.23 grams
100g dark chocolate cocoa solids: 99.23 grams
100g dried cranberries: 113.40 grams
100g or

In [71]:
df

,Recipe Name,Ingredients,Instructions,Image_Name,Calories,Macro_Nutrients,Micro_Nutrients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher sal...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,5243.84,"[(Carbs, 9.78), (Fats, 365.14), (Fiber, 3.4), ...","[(Calcium, 0.36), (Cholesterol, 1.83), (Iron, ..."
2,Thanksgiving Mac and Cheese,1 cup evaporated milk\n1 cup whole milk\n1 tsp...,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,134.29,"[(Carbs, 19.73), (Fats, 2.47), (Fiber, 1.53), ...","[(Calcium, 0.33), (Cholesterol, 0.01), (Magnes..."
3,Italian Sausage and Bread Stuffing,"1 (¾- to 1-pound) round Italian loaf, cut into...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,180.00,"[(Carbs, 41.85), (Fats, 0.45), (Fiber, 7.65), ...","[(Calcium, 0.1), (Magnesium, 0.04), (Phosphoru..."
5,Warm Comfort,2 chamomile tea bags\n1½ oz. reposado tequila\...,Place 2 chamomile tea bags in a heatsafe vesse...,warm-comfort-tequila-chamomile-toddy,43.40,"[(Carbs, 10.64), (Fats, 0.06), (Fiber, 0.03), ...","[(Vitamin A (IU), 21.84), (Fructose, 7.84)]"
7,Turmeric Hot Toddy,¼ cup granulated sugar\n¾ tsp. ground turmeric...,"For the turmeric syrup, combine ½ cup hot wate...",turmeric-hot-toddy-claire-sprouse,242.03,"[(Carbs, 62.11), (Fats, 0.1), (Fiber, 0.72), (...","[(Calcium, 0.01), (Magnesium, 0.01), (Phosphor..."
...,...,...,...,...,...,...,...
13494,Ginger-Pecan Roulade with Honey-Glazed Pecans,"1/2 stick (1/4 cup) unsalted butter, melted, p...",Preheat oven to 350°F. Line bottom and sides o...,ginger-pecan-roulade-with-honey-glazed-pecans-...,1475.80,"[(Carbs, 85.2), (Fats, 129.68), (Fiber, 17.28)...","[(Calcium, 0.29), (Iron, 0.01), (Magnesium, 0...."
13496,Brownie Pudding Cake,1 cup all-purpose flour\n2/3 cup unsweetened c...,Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,1764.00,"[(Carbs, 262.8), (Fats, 79.16), (Fiber, 8.04),...","[(Calcium, 0.27), (Cholesterol, 0.01), (Magnes..."
13497,Israeli Couscous with Roasted Butternut Squash...,1 preserved lemon\n1 1/2 pound butternut squas...,Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,807.60,"[(Carbs, 15.6), (Fats, 81.6), (Fiber, 4.44), (...","[(Calcium, 0.02), (Iron, 0.01), (Magnesium, 0...."
13499,Spanakopita,1 stick (1/2 cup) plus 1 tablespoon unsalted b...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,598.75,"[(Carbs, 9.3), (Fats, 47.63), (Net carbs, 9.3)...","[(Calcium, 1.12), (Cholesterol, 0.2), (Magnesi..."


In [72]:
sample_df = recipes_df.head(5)
for idx, row in sample_df.iterrows():
    print(f"\n=== Recipe {idx + 1}: {row['Title']} ===")
    try:
        ingredients_list = ast.literal_eval(row['Cleaned_Ingredients'])
    except Exception as e:
        print(f"Error parsing ingredients: {e}")
        continue

    for item in ingredients_list:
        print(f"\nRaw Ingredient: {item}")
        merged = parse_and_merge_quantities(item)
        if not merged or merged["quantity"] == 0:
            print("  → Skipped: couldn't parse quantity/unit")
            continue

        qty, unit = merged["quantity"], merged["unit"]
        ingr_raw = merged["ingredient"]
        ingr_cleaned = clean_ingredient(ingr_raw)
        ingr_cleaned = remove_descriptive_phrases(ingr_cleaned)
        grams = qty * unit_to_grams.get(unit, 1.0)

        print(f"  → Parsed: {qty} {unit} of '{ingr_cleaned}' ({grams:.2f} grams)")

        matched = nutrition_df[nutrition_df['Food Name Clean'] == ingr_cleaned]
        if matched.empty and ingr_cleaned.endswith('s'):
            matched = nutrition_df[nutrition_df['Food Name Clean'] == ingr_cleaned[:-1]]

        if matched.empty:
            print("  ❌ No match in nutrition database.")
        else:
            print("  ✅ Matched in nutrition database.")



=== Recipe 1: Miso-Butter Roast Chicken With Acorn Squash Panzanella ===

Raw Ingredient: 1 (3½–4-lb.) whole chicken
  → Parsed: 3.75 lb of 'chicken' (1701.00 grams)
  ❌ No match in nutrition database.

Raw Ingredient: 2¾ tsp. kosher salt, divided, plus more
  → Parsed: 2.75 tsp of 'kosher salt' (11.55 grams)
  ❌ No match in nutrition database.

Raw Ingredient: 2 small acorn squash (about 3 lb. total)
  → Parsed: 3.0 lb of 'acorn squash' (1360.80 grams)
  ❌ No match in nutrition database.

Raw Ingredient: 2 Tbsp. finely chopped sage
  → Parsed: 2.0 tbsp of 'sage' (28.00 grams)
  ❌ No match in nutrition database.

Raw Ingredient: 1 Tbsp. finely chopped rosemary
  → Parsed: 1.0 tbsp of 'rosemary' (14.00 grams)
  ✅ Matched in nutrition database.

Raw Ingredient: 6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature
  → Parsed: 9.0 tbsp of 'butter' (126.00 grams)
  ✅ Matched in nutrition database.

Raw Ingredient: ¼ tsp. ground allspice
  → Parsed: 0.25 tsp of 'allspice' (1.05 gr

In [73]:
zero_calories_count = (df['Calories'] == 0).sum()
print(f"Number of recipes with zero calories: {zero_calories_count}")


Number of recipes with zero calories: 0


In [74]:
df.describe(include='all')


,Recipe Name,Ingredients,Instructions,Image_Name,Calories,Macro_Nutrients,Micro_Nutrients
count,7176,7176,7176,7176,7176.000000,7176,7176
unique,7116,7175,7175,7163,NaN,3312,2950
top,Potato Latkes,1½ oz. vodka\n¾ oz. Kahlúa\n¾ oz. heavy cream,Combine all the ingredients in a cocktail shak...,#NAME?,NaN,"[(Carbs, 120.0), (Net carbs, 120.0), (Sugar, 1...",[]
freq,5,2,2,14,NaN,216,948
mean,NaN,NaN,NaN,NaN,1160.576878,NaN,NaN
std,NaN,NaN,NaN,NaN,17724.186979,NaN,NaN
min,NaN,NaN,NaN,NaN,0.230000,NaN,NaN
25%,NaN,NaN,NaN,NaN,100.800000,NaN,NaN
50%,NaN,NaN,NaN,NaN,459.600000,NaN,NaN
75%,NaN,NaN,NaN,NaN,928.800000,NaN,NaN
